In [27]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

In [28]:
df = pd.read_csv("final_features.csv")
df.head()

,case_id,malignancy,original_shape_LeastAxisLength_0,original_shape_LeastAxisLength_1,original_shape_LeastAxisLength_2,original_shape_LeastAxisLength_3,original_shape_LeastAxisLength_4,original_shape_LeastAxisLength_5,original_shape_LeastAxisLength_6,original_shape_LeastAxisLength_7,...,volume_22,volume_23,volume_24,volume_25,volume_26,volume_27,volume_28,volume_29,volume_30,volume_31
0,LIDC-IDRI-0001_12_84,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LIDC-IDRI-0001_12_85,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LIDC-IDRI-0001_12_86,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LIDC-IDRI-0001_12_87,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LIDC-IDRI-0002_13_88,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def avg_and_std(scores):
    return np.mean(scores), np.std(scores)


def k_fold_cv(model, df, metric_funcs, n_splits=5, pca_components=None):
    """
    Perform k-fold cross-validation on a model.

    Parameters:
    - model: The machine learning model to train and evaluate.
    - df: The pandas DataFrame containing the dataset. Assumes the first column is an identifier and the
          second column is the target variable.
    - metric_funcs: List of metric functions to evaluate, e.g., [roc_auc_score].
    - n_splits: Number of folds for k-fold cross-validation.
    - pca_components: Number of principal components for PCA dimensionality reduction (optional).

    Returns:
    - results: A dictionary containing the metric scores for each fold.
    """
    X = df.iloc[:, 2:].values  # Features 
    y = df.iloc[:, 1].values   # Target 

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    scores = {metric_func.__name__: [] for metric_func in metric_funcs}

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        if pca_components is not None:
            pca = PCA(n_components=pca_components)
            X_train = pca.fit_transform(X_train)
            X_test = pca.transform(X_test)
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict_proba(X_test)[:, 1]
        
        for metric_func in metric_funcs:
            score = metric_func(y_test, y_pred)
            scores[metric_func.__name__].append(score)
    
    return scores

In [ ]:
best_auc = 0
best_params = {}

# Parameters combination test
for crit in ["gini", "entropy", "log_loss"]:
    for n_est in range(25, 201, 25):
        for m_depth in range(5, 106, 10):
            for m_samples_leaf in range(5, 26, 5):
                params = {
                    'n_estimators': n_est,
                    'max_depth': m_depth,
                    'min_samples_leaf': m_samples_leaf,
                    'criterion': crit
                }
                
                rf_model = RandomForestClassifier(**params)
                
                score = k_fold_cv(model=rf_model, df=df, metric_funcs=[roc_auc_score], pca_components=100)
                avg_auc, std_auc = avg_and_std(np.array(score["roc_auc_score"]))
                
                if avg_auc > best_auc:
                    best_auc = avg_auc
                    best_params = params
                    print("New best parameter combination found:")
                else:
                    print("parameter combination tested")
                for parameter, value in best_params.items():
                    print(f"\t{parameter}: {value}")
                print(f"Average AUC: {best_auc * 100:.9f} %\n")

print("Best parameter combination:")
for parameter, value in best_params.items():
    print(f"\t{parameter}: {value}")
print(f"Best Average AUC: {best_auc}")

In [ ]:
print("Best parameter combination:")
for parameter, value in best_params.items():
    print(f"\t{parameter}: {value}")
print(f"Best Average AUC: {best_auc}")